In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
IP = "192.168.1.1"  # rember to set the real IP!
cores = 8

In [ ]:
spark_session = SparkSession.builder\
        .master(f"spark://{IP}:7077") \
        .appName("ECAtemp")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", cores)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

sc = spark_session.sparkContext

sc.setLogLevel("ERROR")

All the data should have been extracted to HDFS. Load the data:

In [ ]:
data_root = f"hdfs://{IP}:9000/data/"
rdds = {}

for data_id in ("tx", "tn"):
	data_dir = data_root + data_id
	rdds[data_id] = sc.textFile(data_dir)

Read the data skipping the info lines:

In [ ]:
for data_id in ("tx", "tn"):
	rdds[data_id].zipWithIndex().\
		filter(lambda x: x[1] >= 20).\
		map(lambda x: x[0]).\
		toDF()